# LLM Agents Walkthrough

This is a  tutorial on building LLM agents! This notebook will take you through a building basic agent concepts to sophisticated multi-agent systems.

## What You'll Learn

1. **Basic Agent Architecture**: Understanding the core components of LLM agents
2. **Specialized Agents**: Creating agents with specific capabilities (fact extraction, sentiment analysis)
3. **Tool Integration**: Enabling agents to use external tools (web search, Wikipedia)
4. **Agent Orchestration**: Coordinating multiple agents to work together
5. **Real-world Applications**: Practical examples and best practices

## Prerequisites
- Google Gemini API key (set in GEMINI_API_KEY environment variable)
- Brave Search API key for web search functionality

In [88]:
import json
import re
import os
from typing import Dict
from dataclasses import dataclass
import google.generativeai as genai
from dotenv import load_dotenv
from pprint import pp
import requests
import wikipediaapi
import time

load_dotenv()

True

## Configuring the LLM

Now let's configure our Large Language Model (Google Gemini). This is the brain of our agents.

**Important**: Make sure you have your `GEMINI_API_KEY` set in your environment variables or `.env` file. You can get a free API key from [Google AI Studio](https://makersuite.google.com/app/apikey).

The configuration below will:
- Load the API key from environment variables
- Initialize the Gemini model (we're using gemini-2.0-flash for its speed and capability)
- Confirm successful setup

In [89]:
api_key = os.getenv('GEMINI_API_KEY')
if not api_key:
    print("GEMINI_API_KEY environment variable is not set")
else:
    genai.configure(api_key=api_key)
    gemini = genai.GenerativeModel('gemini-2.0-flash')
    print("Gemini model configured successfully")

Gemini model configured successfully


## Testing Basic LLM Interaction

Before we build agents, let's test our basic LLM setup with a simple query. This will help us understand the foundation upon which we'll build our agent system.

We'll ask about AI trends to see how the model responds to a contemporary topic.

In [90]:
query = "What are the trends in AI impact on job markets in 2025?"

In [91]:
response = gemini.generate_content(query)
print(response.text)

Okay, let's break down the projected trends in AI's impact on the job market in 2025.  It's important to remember that these are forecasts, and the actual reality could vary. However, we can draw from current data, expert opinions, and industry reports to paint a likely picture.

**Overall Impact: A Mixed Bag - Job Displacement & Creation**

The consensus is that AI will lead to both job displacement *and* job creation. It's not simply a story of robots taking all our jobs.  The key is understanding *which* jobs will be affected and *how*.

**1. Job Displacement (Automation of Existing Tasks):**

*   **Repetitive, Rule-Based Tasks:** This is the most obvious area. AI and automation will continue to replace jobs involving routine data entry, basic customer service (e.g., chatbots), simple manufacturing tasks, and repetitive administrative functions. Examples include:
    *   **Data Entry Clerks:**  AI-powered OCR and data extraction tools will continue to automate this.
    *   **Assemb

## Building Our First Agent: The BaseAgent Class

Now we'll create our first agent! An **agent** is essentially a wrapper around an LLM that gives it specific instructions and behavior.

### What Makes This an Agent?

1. **Identity**: Each agent has a name and specific role
2. **Instructions**: Pre-defined system prompts that shape behavior
3. **Consistent Interface**: A standard way to process queries
4. **Specialization**: Each agent can be tailored for specific tasks

The `BaseAgent` class will serve as the foundation for all our specialized agents. It demonstrates the core pattern: **LLM + Instructions = Agent**.

### 🏗️ Basic Agent Architecture

```
┌─────────────────────────────────────────┐
│                BaseAgent                │
├─────────────────────────────────────────┤
│  Properties:                            │
│  • name: str                            │
│  • instructions: str                    │
├─────────────────────────────────────────┤
│  Methods:                               │
│  • process(query) → response            │
└─────────────────────────────────────────┘
                    │
                    ▼
┌─────────────────────────────────────────┐
│              Query Flow                 │
├─────────────────────────────────────────┤
│  User Query                             │
│       │                                 │
│       ▼                                 │
│  Instructions + Query                   │
│       │                                 │
│       ▼                                 │
│  LLM (Gemini)                           │
│       │                                 │
│       ▼                                 │
│  Processed Response                     │
└─────────────────────────────────────────┘
```

In [92]:
class BaseAgent:
    """Simple base agent that wraps LLM calls with specific instructions"""
    
    def __init__(self, name: str, instructions: str):
        self.name = name
        self.instructions = instructions
    
    def process(self, query: str) -> str:
        """Process query with agent-specific instructions"""
        full_prompt = f"{self.instructions}\n\nQuery: {query}"
        return gemini.generate_content(full_prompt).text

## Creating Specialized Agents

Now let's create our first specialized agents! Each agent will have different instructions that make it excel at specific tasks:

- **FactExtractor**: Focuses on extracting factual information and statistics
- **SentimentAnalyzer**: Specializes in analyzing emotions and sentiment
- **Summarizer**: Excels at creating concise summaries

This demonstrates how the same LLM can be given different "personalities" and capabilities through prompt engineering.

### 🎭 Specialized Agent Architecture

```
                    BaseAgent (Foundation)
                          │
        ┌─────────────────┼─────────────────┐
        │                 │                 │
        ▼                 ▼                 ▼
┌──────────────┐ ┌─────────────────┐    ┌──────────────┐
│FactExtractor │ │SentimentAnalyzer│    │ Summarizer   │
├──────────────┤ ├─────────────────┤    ├──────────────┤
│Instructions: │ │Instructions:    │    │Instructions: │
│"Extract only │ │"Analyze         │    │"Generate     │
│ factual info │ │ sentiment       │    │ concise      │
│ and stats"   │ │ and emotion"    │    │ summary"     │
└──────────────┘ └─────────────────┘    └──────────────┘
        │                 │                 │
        ▼                 ▼                 ▼
┌──────────────┐ ┌──────────────┐ ┌──────────────┐
│   Output:    │ │   Output:    │ │   Output:    │
│ • Facts      │ │ • Positive/  │ │ • Key Points │
│ • Statistics │ │   Negative   │ │ • Summary    │
│ • Numbers    │ │ • Emotions   │ │ • Overview   │
└──────────────┘ └──────────────┘ └──────────────┘

Same Input → Different Processing → Different Outputs
```

In [93]:
fact_extractor = BaseAgent("FactExtractor", "Extract only factual information and statistics.")
sentiment_analyzer = BaseAgent("SentimentAnalyzer", "Analyze sentiment and emotional tone.")
summarizer = BaseAgent("Summarizer", "Generate concise summary.")

## 6. Testing Our Specialized Agents

Let's test each of our specialized agents with the same query to see how their different instructions affect their responses. Notice how each agent will focus on different aspects of the same information!

In [94]:
print(f"Facts: {fact_extractor.process(query)}")

Facts: Based on various reports and studies, here's a summary of potential factual information and statistics regarding AI's impact on job markets in 2025:

*   **Job Displacement:** Studies estimate that AI could displace millions of jobs. Some estimate displacement ranging from tens of millions to over 85 million jobs globally.

*   **Job Creation:** AI is also expected to create new jobs. The number is debated, but some reports suggest AI could create significantly more jobs than it displaces (e.g., over 97 million new jobs).

*   **Skill Shifts:** There is an expected shift in required skills. Increased demand for skills such as AI and Machine Learning Specialists, Data Analysts and Scientists, Big Data Specialists, Digital Marketing and Strategy Specialists, and Process Automation Specialists.

*   **Automation Potential:** A significant percentage of current job tasks could be automated by AI. Some reports suggest up to 40-50% of current work activities could be automated using c

In [95]:
print(f"Sentiment: {sentiment_analyzer.process(query)}")  

Sentiment: The query "What are the trends in AI impact on job markets in 2025?" is **neutral** in sentiment and emotional tone.

Here's why:

*   **No inherent positive or negative language:** The question simply seeks information about a specific topic (AI's impact on job markets). It doesn't express an opinion or emotion about this topic.
*   **Objective inquiry:** The question is phrased in a way that aims to gather objective information or predictions about future trends.
*   **Absence of emotional keywords:** There are no words or phrases that evoke feelings like joy, fear, anger, sadness, etc.

The query is purely **informational** and aims to understand potential changes in the job market due to AI.



In [96]:
print(f"Summary: {summarizer.process(query)}")

Summary: In 2025, AI is projected to:

*   **Increase demand for AI-related roles:** Data scientists, AI engineers, and machine learning specialists will be highly sought after.
*   **Reshape existing roles:** Workers will need to adapt and acquire skills to work alongside AI systems, focusing on areas like creativity, critical thinking, and complex problem-solving.
*   **Automate some tasks:** Routine and repetitive jobs are most vulnerable to automation, potentially leading to job displacement in certain sectors.
*   **Create new job categories:** AI will spur innovation and generate entirely new types of jobs that don't currently exist.
*   **Widen the skills gap:** Demand for AI-related skills will outstrip supply, requiring investment in education and training programs.



## 7. Structured Output: JSON Processing Utility

As we build more sophisticated agents, we'll need them to return structured data instead of just text. This utility function helps us:

1. **Extract JSON from LLM responses**: LLMs often wrap JSON in markdown code blocks
2. **Handle parsing errors gracefully**: Provides clear error messages when JSON is malformed
3. **Enable structured agent communication**: Allows agents to return dictionaries instead of just strings

In [97]:
def extract_and_parse_json(text_response):
    """
    Extracts a JSON string from a text that might be wrapped in Markdown code blocks
    (e.g., ```json...```) and then parses it into a Python dictionary.

    Args:
        text_response (str): The raw string response from the AI model.

    Returns:
        dict: The parsed JSON object.
        None: If no valid JSON is found or if parsing fails.
    """
    # Regex to find content inside ```json ... ``` or just ``` ... ```
    # It tries to find '```json' first, then falls back to '```'
    json_match = re.search(r'```(?:json)?(?s)(.*?)```', text_response)

    if json_match:
        # Extract the content within the backticks and strip any leading/trailing whitespace
        json_string = json_match.group(1).strip()
    else:
        # If no markdown block found, assume the entire response is the JSON string
        json_string = text_response.strip()

    try:
        # Attempt to parse the cleaned string as JSON
        parsed_data = json.loads(json_string)
        return parsed_data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Problematic JSON string: '{json_string}'")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during JSON processing: {e}")
        return None

## Advanced Agent Architecture: Structured Output Agents

Now let's build more sophisticated agents that return structured data! This `FactExtractorAgent` demonstrates several advanced concepts:

### Key Improvements:
1. **Structured Output**: Returns JSON with organized facts, entities, and statistics
2. **Detailed Instructions**: More specific prompts that guide the LLM to produce consistent output
3. **Error Handling**: Uses our JSON parsing utility to handle malformed responses
4. **Specialized Methods**: Different methods for different types of operations

This pattern makes agents more reliable and their outputs more useful for downstream processing.

### 🏗️ Advanced Agent Architecture

```
┌─────────────────────────────────────────────────────────┐
│                 FactExtractorAgent                      │
├─────────────────────────────────────────────────────────┤
│  Input: Raw Text                                        │
│       │                                                 │
│       ▼                                                 │
│  ┌─────────────────────────────────────────────────┐    │
│  │        Detailed JSON Instructions               │    │
│  │  • Must return JSON format                      │    │
│  │  • Specific keys required                       │    │
│  │  • Example format provided                      │    │
│  └─────────────────────────────────────────────────┘    │
│       │                                                 │
│       ▼                                                 │
│  ┌─────────────────────────────────────────────────┐    │
│  │              LLM Processing                     │    │
│  └─────────────────────────────────────────────────┘    │
│       │                                                 │
│       ▼                                                 │
│  ┌─────────────────────────────────────────────────┐    │
│  │           JSON Parser                           │    │
│  │  • Extract from markdown blocks                 │    │
│  │  • Handle parsing errors                        │    │
│  │  • Return structured dict                       │    │
│  └─────────────────────────────────────────────────┘    │
│       │                                                 │
│       ▼                                                 │
│  ┌─────────────────────────────────────────────────┐    │
│  │          Structured Output                      │    │
│  │  {                                              │    │
│  │    "facts": ["fact1", "fact2"],                 │    │
│  │    "entities": ["entity1", "entity2"],          │    │
│  │    "statistics": ["40%", "2024"],               │    │
│  │    "summary": "Brief overview",                 │    │
│  │    "key_points": ["point1", "point2"]           │    │
│  │  }                                              │    │
│  └─────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────┘

Basic Agent → Advanced Agent with Structured Output
```

In [98]:
class FactExtractorAgent(BaseAgent):
    """Agent specialized in extracting facts and statistics"""
    
    def __init__(self):
        super().__init__("FactExtractor", "Extract facts and statistics from text. Return structured data.")
    
    def extract_facts(self, text: str) -> Dict:
        """Extract structured facts from text"""
        prompt = (
            f"Extract all salient factual information from the following text: '{text}'. "
            "Identify distinct factual statements, key entities mentioned, and provide a concise summary of the factual content. "
            "Your response MUST be a JSON object containing ONLY the following keys: "
            "'facts' (as a list of strings, each a distinct factual statement), "
            "'entities' (as a list of relevant entities mentioned, e.g., people, organizations, dates, locations), "
            "'statistics' (a list of numerical data or statistics extracted), "
            "'summary' (a brief summary of the extracted facts), "
            "'key_points' (a list of key points derived from the facts). "
            "DO NOT include any additional text, explanations, or Markdown formatting (like ```json)."
            f"\n\nExample desired format (values are illustrative): {{"
            f"\"facts\": ["
            f"    \"The Eiffel Tower is located in Paris, France.\","
            f"    \"It was completed in 1889.\","
            f"    \"Gustave Eiffel's company designed it.\""
            f"],"
            f"\"entities\": [\"Eiffel Tower\", \"Paris\", \"France\", \"1889\", \"Gustave Eiffel\"],"
            f"\"summary\": \"Key facts about the Eiffel Tower, including its location, completion date, and designer.\","
            f"}}"
        )
        response = self.process(prompt)
        return extract_and_parse_json(response)

### Advanced Sentiment Analysis Agent

Similarly, here's our enhanced sentiment analyzer that returns structured sentiment analysis:

**Features:**
- **Confidence Scoring**: How certain the agent is about its analysis
- **Tone Detection**: Identifies specific emotions beyond just positive/negative
- **Justification**: Explains why it reached its conclusion
- **Consistent Format**: Always returns the same JSON structure

This makes sentiment analysis results much more actionable and interpretable.

In [99]:
class SentimentAnalyzerAgent(BaseAgent):
    """Agent specialized in sentiment analysis"""
    
    def __init__(self):
        super().__init__("SentimentAnalyzer", "Analyze sentiment and return positive/negative/neutral.")
    
    def analyze_sentiment(self, text: str) -> Dict:
        """Analyze sentiment with confidence score"""
        prompt = (
            f"Analyze the sentiment of the following text: '{text}'. "
            "Return the primary sentiment as 'positive', 'negative', or 'neutral', "
            "along with an overall confidence score. "
            "Also, identify the specific emotional 'tone(s)' present (e.g., 'joy', 'anger', 'sadness', 'excitement') "
            "and provide a 'justification' explaining why that sentiment was assigned, "
            "citing specific parts of the text if possible. "
            "Your response MUST be a JSON object containing ONLY the following keys: "
            "'sentiment', 'confidence', 'tone' (as a list of strings), and 'justification'. "
            "DO NOT include any additional text, explanations, or Markdown formatting (like ```json)."
            f"\n\nExample desired format (values are illustrative): {{"
            f"\"sentiment\": \"positive\", "
            f"\"confidence\": 0.92, "
            f"\"tone\": [\"joy\", \"excitement\"], "
            f"\"justification\": \"The user expressed enthusiasm with phrases like 'absolutely loved it' and 'highly recommend'.\""
            f"}}"
        )
        response = self.process(prompt)
        return extract_and_parse_json(response)

## 9. Testing Advanced Agents

Let's instantiate our advanced agents and test them with structured output. Notice how much more useful and organized the results are compared to our basic agents!

In [100]:
fact_agent = FactExtractorAgent()
sentiment_agent = SentimentAnalyzerAgent()

### Sample Text for Testing

Let's use a sample text that contains both factual information and emotional content to test our agents:

In [101]:
print("🔍 Specialized extraction:")
sample_text = "AI adoption increased 40% in 2024. This creates positive opportunities but requires adaptation."
facts = fact_agent.extract_facts(sample_text)
sentiment = sentiment_agent.analyze_sentiment(sample_text)
print(f"*****Extracted Facts******")
pp(facts, indent=4)
print()
print(f"*****Sentiment Analysis******")
pp(sentiment, indent=4)

🔍 Specialized extraction:
*****Extracted Facts******
{   'facts': [   'AI adoption increased 40% in 2024.',
                 'AI adoption creates positive opportunities.',
                 'AI adoption requires adaptation.'],
    'entities': ['AI', '2024'],
    'statistics': ['40%'],
    'summary': 'AI adoption saw a 40% increase in 2024, creating opportunities '
               'but requiring adaptation.',
    'key_points': [   'AI adoption is growing.',
                      'Growth presents both benefits and challenges.']}

*****Sentiment Analysis******
{   'sentiment': 'positive',
    'confidence': 0.75,
    'tone': ['optimism', 'anticipation'],
    'justification': "The text highlights a significant increase ('AI adoption "
                     "increased 40%') and explicitly mentions 'positive "
                     "opportunities'. While it acknowledges the need for "
                     "'adaptation', the primary focus is on the positive "
                     'aspects of this 

## Agent Coordination Patterns

Now let's explore different ways to coordinate multiple agents. There are two main patterns:

### 1. Independent (Parallel) Agents
- All agents work on the same input simultaneously
- Fast execution, no dependencies
- Good for getting different perspectives on the same data

### 2. Sequential (Pipeline) Agents  
- Agents work in sequence, with each building on the previous
- More sophisticated processing, but slower
- Good for complex analysis where later steps depend on earlier results

Let's implement both patterns and see the differences!

### 🔄 Agent Coordination Architecture

#### Pattern 1: Independent (Parallel) Processing
```
                    Input Text
                         │
        ┌────────────────┼────────────────┐
        │                │                │
        ▼                ▼                ▼
┌──────────────┐ ┌─────────────────┐ ┌──────────────┐
│ FactExtractor│ │SentimentAnalyzer│ │ Summarizer   │
│              │ │                 │ │              │
│   (Agent 1)  │ │   (Agent 2)     │ │   (Agent 3)  │
└──────────────┘ └─────────────────┘ └──────────────┘
        │                │                │
        ▼                ▼                ▼
┌──────────────┐ ┌──────────────┐ ┌──────────────┐
│   Facts      │ │  Sentiment   │ │   Summary    │
│   Output     │ │   Output     │ │   Output     │
└──────────────┘ └──────────────┘ └──────────────┘
        │                │                │
        └────────────────┼────────────────┘
                         ▼
                 Combined Results
```

#### Pattern 2: Sequential (Pipeline) Processing
```
Input Text
    │
    ▼
┌──────────────┐
│ FactExtractor│  Step 1: Extract Facts
│   (Agent 1)  │
└──────────────┘
    │
    ▼ (Facts Output)
┌──────────────┐
│SentimentAnalyzer Step 2: Analyze Sentiment of Facts
│   (Agent 2)  │
└──────────────┘
    │
    ▼ (Facts + Sentiment)
┌──────────────┐
│ Summarizer   │  Step 3: Enhanced Summary with Context
│   (Agent 3)  │
└──────────────┘
    │
    ▼
Enhanced Result (Each step builds on previous)
```

In [102]:
def run_independent_agents(text: str):
    print("Running agents in parallel...")
    
    # All agents work on same input independently
    facts = fact_agent.extract_facts(text)
    sentiment = sentiment_agent.analyze_sentiment(text)
    summary = summarizer.process(text)
    
    return {
        "facts": facts,
        "sentiment": sentiment,  
        "summary": summary
    }

def run_sequential_agents(text: str):
    print("Running agents sequentially...")
    
    # Step 1: Extract facts first
    facts = fact_agent.extract_facts(text)
    print(f"Step 1 - Facts extracted: {len(facts['statistics'])} statistics found")
    
    # Step 2: Analyze sentiment of extracted facts specifically
    facts_text = " ".join(facts['key_points'])
    sentiment = sentiment_agent.analyze_sentiment(facts_text)
    print(f"Step 2 - Sentiment of facts: {sentiment['sentiment']}")
    
    # Step 3: Create summary using both facts and sentiment
    enhanced_input = f"Facts: {facts_text}. Sentiment: {sentiment['sentiment']}"
    summary = summarizer.process(enhanced_input)
    print(f"Step 3 - Enhanced summary created")
    
    return {
        "facts": facts,
        "sentiment": sentiment,
        "enhanced_summary": summary
    }

### Testing Sequential Agent Coordination

Let's test our sequential agent pattern where each agent builds on the results of the previous one:

In [103]:
dependent_results = run_sequential_agents(sample_text)
print()

Running agents sequentially...
Step 1 - Facts extracted: 1 statistics found
Step 1 - Facts extracted: 1 statistics found
Step 2 - Sentiment of facts: neutral
Step 2 - Sentiment of facts: neutral
Step 3 - Enhanced summary created

Step 3 - Enhanced summary created



In [104]:
dependent_results

{'facts': {'facts': ['AI adoption increased in 2024.',
   'AI adoption increased 40% in 2024.',
   'This creates positive opportunities.',
   'This requires adaptation.'],
  'entities': ['AI', '2024'],
  'statistics': ['40%'],
  'summary': 'AI adoption increased by 40% in 2024, leading to positive opportunities but requiring adaptation.',
  'key_points': ['AI adoption saw a significant increase in 2024.',
   'This increase presents both opportunities and challenges.']},
 'sentiment': {'sentiment': 'neutral',
  'confidence': 0.75,
  'tone': ['objective'],
  'justification': "The text presents a factual statement about AI adoption increasing and then acknowledges both positive ('opportunities') and negative ('challenges') aspects, leading to an overall neutral sentiment. There's no explicit emotional language used."},
 'enhanced_summary': 'AI adoption significantly increased in 2024, presenting both opportunities and challenges.\n'}

## Tool-Enabled Agents: Adding External Capabilities

So far our agents have only worked with the information they receive as input. But what if they need to gather additional information? This is where **tool-enabled agents** come in!

### The ToolRegistry Class

This class provides our agents with external capabilities:

1. **Web Search**: Using Brave Search API to find current information
2. **Wikipedia Search**: Accessing Wikipedia's knowledge base
3. **Dynamic Tool Discovery**: Agents can discover what tools are available
4. **Error Handling**: Graceful handling of API failures

**Key Concept**: Tools extend agent capabilities beyond just text processing, allowing them to interact with the external world.

### 🛠️ Tool Registry Architecture

```
┌─────────────────────────────────────────────────────────┐
│                   ToolRegistry                          │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  ┌─────────────────┐    ┌──────────────────┐            │
│  │   web_search()  │    │wikipedia_search()│            │
│  ├─────────────────┤    ├──────────────────┤            │
│  │ • Query: str    │    │ • Query: str     │            │
│  │ • Max results   │    │ • Sentences      │            │
│  │ • API handling  │    │ • Error handling │            │
│  └─────────────────┘    └──────────────────┘            │
│           │                       │                     │
│           ▼                       ▼                     │
│  ┌─────────────────┐    ┌─────────────────┐             │
│  │ Brave Search    │    │ Wikipedia API   │             │
│  │ Results         │    │ Results         │             │
│  └─────────────────┘    └─────────────────┘             │
└─────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────┐
│              Agent + Tools Integration                  │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Agent Internal Knowledge  +  External Tool Access      │
│           │                           │                 │
│           ▼                           ▼                 │
│  ┌─────────────────┐         ┌─────────────────┐        │
│  │ LLM Reasoning   │         │ Real-time Data  │        │
│  │ • Patterns      │         │ • Current Info  │        │
│  │ • Training Data │         │ • Specific Facts│        │
│  │ • Analysis      │         │ • External APIs │        │
│  └─────────────────┘         └─────────────────┘        │
│           │                           │                 │
│           └───────────┬───────────────┘                 │
│                       ▼                                 │
│              Enhanced Agent Output                      │
└─────────────────────────────────────────────────────────┘
```

In [105]:
class ToolRegistry:
    """Registry of tools that agents can use"""
    
    @classmethod
    def get_available_tools(cls) -> str:
        """Get formatted description of all available tools"""
        tools_description = []
        
        import inspect
        # Use inspect.isfunction since we're using @staticmethod
        for name, method in inspect.getmembers(cls, predicate=inspect.isfunction):
            if not name.startswith('_') and name != 'get_available_tools':  # Skip private methods and self
                # Get the docstring
                doc = inspect.getdoc(method)
                if doc:
                    # Extract the first line as description
                    description = doc.split('\n')[0].strip()
                    
                    # Get function signature
                    sig = inspect.signature(method)
                    params = []
                    for param_name, param in sig.parameters.items():
                        param_type = param.annotation.__name__ if param.annotation != inspect.Parameter.empty else 'any'
                        default = f" = {param.default}" if param.default != inspect.Parameter.empty else ""
                        params.append(f"{param_name}: {param_type}{default}")
                    
                    params_str = ", ".join(params)
                    tools_description.append(f"- {name}({params_str}) - {description}")
        
        return "\n".join(tools_description)

    @staticmethod
    def web_search(query: str, max_results: int = 5) -> Dict:
        """
        Performs a web search using Brave Search API and returns a summary of the top results.

        Requires BRAVE_SEARCH_API_KEY environment variable to be set.

        Args:
            query (str): The search query.
            max_results (int): The maximum number of search results to return (Brave API limit might be 20 for free tier).

        Returns:
            Dict: A dictionary containing the query and a list of search results.
                  Each result includes 'title', 'url', and 'snippet'.
                  Includes an 'error' key if the search fails.
        """
        print(f"Tool Call: web_search(query='{query}', max_results={max_results})")
        
        brave_api_key = os.getenv("BRAVE_SEARCH_API_KEY")
        if not brave_api_key:
            return {"error": "BRAVE_SEARCH_API_KEY environment variable not set. Please get a key from brave.com/search/api/"}

        headers = {
            "X-Subscription-Token": brave_api_key,
            "Accept": "application/json"
        }
        params = {
            "q": query,
            "count": min(max_results, 10), # Brave free tier often limits to 20 results per call
            "search_lang": "en"
        }

        results = []
        try:
            response = requests.get(
                "https://api.search.brave.com/res/v1/web/search",
                headers=headers,
                params=params
            )
            response.raise_for_status() # Raises HTTPError for bad responses (4xx or 5xx)
            data = response.json()

            if 'web' in data and 'results' in data['web']:
                for r in data['web']['results']:
                    results.append({
                        "title": r.get("title", "N/A"),
                        "url": r.get("url", "N/A"),
                        "snippet": r.get("description", "N/A") # Brave API uses 'description' for snippet
                    })
            else:
                return {"query": query, "results": [], "error": "No 'web' or 'results' key found in Brave API response."}

        except requests.exceptions.HTTPError as http_err:
            return {"query": query, "results": [], "error": f"HTTP error occurred: {http_err} - {response.text}"}
        except requests.exceptions.ConnectionError as conn_err:
            return {"query": query, "results": [], "error": f"Connection error: {conn_err}"}
        except requests.exceptions.Timeout as timeout_err:
            return {"query": query, "results": [], "error": f"Timeout error: {timeout_err}"}
        except requests.exceptions.RequestException as req_err:
            return {"query": query, "results": [], "error": f"An unexpected request error occurred: {req_err}"}
        except Exception as e:
            return {"query": query, "results": [], "error": f"An unexpected error occurred during Brave search: {e}"}

        return {
            "query": query,
            "results": results
        }
    
    @staticmethod
    def wikipedia_search(query: str, sentences: int = 3) -> Dict:
        """
        Searches Wikipedia for a query and returns a summary and URL of the page.

        Args:
            query (str): The search query for Wikipedia.
            sentences (int): The number of sentences to retrieve for the summary.

        Returns:
            Dict: A dictionary containing the query, title, summary, and URL of the Wikipedia page.
                  Returns 'error' if the page is not found.
        """
        print(f"Tool Call: wikipedia_search(query='{query}', sentences={sentences})")
        # It's good practice to provide a user_agent to Wikipedia API
        # Replace 'your_email@example.com' with an actual contact email.
        wiki_wiki = wikipediaapi.Wikipedia(language='en', user_agent="AgenticFrameworkTutorial/1.0 (sajjad.riaj@gmail.com)")
        
        page = wiki_wiki.page(query)

        if page.exists():
            # Truncate summary gracefully if it's too long
            summary_text = page.summary
            # A rough heuristic for sentences to words: ~15-20 words per sentence
            if len(summary_text.split()) > sentences * 20: 
                summary_text = ' '.join(summary_text.split()[:sentences * 20]) + "..."

            return {
                "query": query,
                "title": page.title,
                "summary": summary_text,
                "url": page.fullurl
            }
        else:
            return {
                "query": query,
                "error": f"No Wikipedia page found for '{query}'."
            }

## Test the tools

Now lets test the tools and their functionalities.

In [106]:
# Make sure to set your BRAVE_SEARCH_API_KEY environment variable before running!

print("--- Testing web_search (Brave API) ---")
web_results = ToolRegistry.web_search("latest AI trends 2025", max_results=3)
print(web_results)

time.sleep(3)

web_results_short = ToolRegistry.web_search("quantum computing breakthroughs", max_results=1)
print(web_results_short)

time.sleep(3)

print("\n--- Testing wikipedia_search ---")
wiki_results = ToolRegistry.wikipedia_search("Artificial General Intelligence")
pp(wiki_results, indent=4)

wiki_results_short = ToolRegistry.wikipedia_search("Generative AI", sentences=1)
pp(wiki_results_short, indent=4)

wiki_not_found = ToolRegistry.wikipedia_search("NonExistentTopicForSure12345")
pp(wiki_not_found, indent=4)

--- Testing web_search (Brave API) ---
Tool Call: web_search(query='latest AI trends 2025', max_results=3)
{'query': 'latest AI trends 2025', 'results': [{'title': '6 AI trends you’ll see more of in 2025', 'url': 'https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/', 'snippet': '“We’ll start to see these tools ... drugs,” Llorens says.  · In 2025, one trend is certain: <strong>AI will continue to drive innovation and unlock new potential for people and organizations around the globe</strong>....'}, {'title': "AI Patent Trends Signal Tomorrow's Technologies - Patent - China", 'url': 'https://www.mondaq.com/china/patent/1643334/ai-patent-trends-signal-tomorrows-technologies', 'snippet': 'Artificial intelligence (<strong>AI</strong>) is the ultimate buzzword at the moment. But from a patent-based perspective, it is more than just the new fad: it is a pointer to technological developments and signals...'}, {'title': 'Latest AI Breakthroughs and News: May, J

## 12. Advanced Agent Architecture: Tool-Enabled Agents

Now let's build our most sophisticated agents yet! These agents can:

1. **Decide** which tools to use based on the query
2. **Execute** the tools with appropriate parameters
3. **Synthesize** the results into a comprehensive response

### The Three-Step Process:

1. **Tool Decision**: The agent analyzes the query and decides which tools it needs
2. **Tool Execution**: The agent calls the selected tools with generated parameters
3. **Result Synthesis**: The agent combines tool outputs with its own analysis

This is a powerful pattern that mimics how humans solve complex problems by gathering information from multiple sources.

### 🧠 Advanced Tool-Enabled Agent Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│              FactExtractorAgentToolsEnabled                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Step 1: TOOL DECISION                                          │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │  Query Analysis                                         │    │
│  │  • What information is needed?                          │    │
│  │  • Which tools can provide it?                          │    │
│  │  • What parameters to use?                              │    │
│  │                                                         │    │
│  │  Input: "Tell me about iPhone 15"                       │    │
│  │  Decision: Use web_search("iPhone 15 specs")            │    │
│  └─────────────────────────────────────────────────────────┘    │
│                              │                                  │
│                              ▼                                  │
│  Step 2: TOOL EXECUTION                                         │
│  ┌──────────────────────────────────────────────────────────┐   │
│  │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐       │   │
│  │  │ web_search  │  │ wikipedia   │  │  Other      │       │   │
│  │  │ (if needed) │  │ (if needed) │  │  Tools      │       │   │
│  │  └─────────────┘  └─────────────┘  └─────────────┘       │   │
│  │         │                │                │              │   │
│  │         ▼                ▼                ▼              │   │
│  │  ┌─────────────────────────────────────────────────────┐ │   │
│  │  │           Tool Results Collection                   │ │   │
│  │  │  • Web search results                               │ │   │
│  │  │  • Wikipedia summaries                              │ │   │
│  │  │  • Error handling                                   │ │   │
│  │  └─────────────────────────────────────────────────────┘ │   │
│  └──────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              ▼                                  │
│  Step 3: RESULT SYNTHESIS                                       │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │  Original Query + Tool Results + Agent Analysis         │    │
│  │                        │                                │    │
│  │                        ▼                                │    │
│  │  ┌─────────────────────────────────────────────────┐    │    │
│  │  │          LLM Synthesis                          │    │    │
│  │  │  • Combine information                          │    │    │
│  │  │  • Extract facts                                │    │    │
│  │  │  • Cite sources                                 │    │    │
│  │  │  • Structure output                             │    │    │
│  │  └─────────────────────────────────────────────────┘    │    │
│  │                        │                                │    │
│  │                        ▼                                │    │
│  │  ┌─────────────────────────────────────────────────┐    │    │
│  │  │        Comprehensive Response                   │    │    │
│  │  │  {                                              │    │    │
│  │  │    "facts": [...],                              │    │    │
│  │  │    "sources_used": [...],                       │    │    │
│  │  │    "confidence": 0.95,                          │    │    │
│  │  │    "summary": "...",                            │    │    │
│  │  │    "tool_decision": {...},                      │    │    │
│  │  │    "tool_results": {...}                        │    │    │
│  │  │  }                                              │    │    │
│  │  └─────────────────────────────────────────────────┘    │    │
│  └─────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────┘

DECIDE → EXECUTE → SYNTHESIZE
Human-like Problem Solving
```

In [107]:
class FactExtractorAgentToolsEnabled(BaseAgent):
    """Fact agent that can decide to use tools"""
    
    def __init__(self):
        super().__init__(
            "FactAgent", 
            "You are a fact extraction agent. You can use web search and Wikipedia tools when needed."
        )
        self.tools = ToolRegistry()
    
    def handle_query(self, query: str) -> Dict:
        """Main method: decide tools → execute tools → synthesize"""
        print(f"\n🤖 [{self.name}] Received query: {query}")
        
        # Step 1: Decide which tools to use and generate params
        tool_decision = self._decide_tools(query)
        print(f"🔧 [{self.name}] Tool decision: {tool_decision}")
        
        # Step 2: Execute the tools
        tool_results = self._execute_tools(tool_decision)
        print(f"📊 [{self.name}] Tool execution complete")
        
        # Step 3: Synthesize final result
        final_result = self._synthesize_result(query, tool_decision, tool_results)
        print(f"✅ [{self.name}] Result synthesized")
        
        return final_result
    
    def _decide_tools(self, query: str) -> Dict:
        """Step 1: Decide which tools to use and generate parameters"""
        print(f"🧠 [{self.name}] Deciding which tools to use...")
        
        # Get available tools from ToolRegistry
        available_tools = ToolRegistry.get_available_tools()
        
        decision_prompt = f"""
        Query: {query}
        
        Available tools:
        {available_tools}
        
        Decide which tools to use for this query and generate the parameters.
        
        Respond with JSON:
        {{
            "tools_to_use": [
                {{
                    "tool": "tool_name",
                    "params": {{"param1": "value1", "param2": "value2"}}
                }}
            ],
            "reasoning": "why these tools and params"
        }}
        
        Use empty list [] if no tools needed.
        """
        
        response = self.process(decision_prompt)
        return extract_and_parse_json(response)
    
    def _execute_tools(self, tool_decision: Dict) -> Dict:
        """Step 2: Execute the decided tools with their parameters"""
        print(f"⚡ [{self.name}] Executing tools...")
        
        tool_results = {}
        tools_to_use = tool_decision.get("tools_to_use", [])
        
        for i, tool_call in enumerate(tools_to_use):
            tool_name = tool_call.get("tool")
            params = tool_call.get("params", {})
            
            print(f"🔍 [{self.name}] Executing {tool_name} with params: {params}")
            
            # Use getattr to call tools dynamically
            if hasattr(self.tools, tool_name):
                tool_method = getattr(self.tools, tool_name)
                try:
                    result = tool_method(**params)
                except TypeError as e:
                    result = {"error": f"Invalid parameters for {tool_name}: {e}"}
            else:
                result = {"error": f"Tool {tool_name} not found in ToolRegistry"}
            
            tool_results[f"{tool_name}_{i}"] = result
            print(f"📋 [{self.name}] {tool_name} completed")
        
        return tool_results
    
    def _synthesize_result(self, original_query: str, tool_decision: Dict, tool_results: Dict) -> Dict:
        """Step 3: Synthesize final result from tools and original query"""
        print(f"🧬 [{self.name}] Synthesizing final result...")
        
        synthesis_prompt = f"""
        Original query: {original_query}
        
        Tool decision made: {json.dumps(tool_decision, indent=2)}
        
        Tool results: {json.dumps(tool_results, indent=2)}
        
        Now extract facts and synthesize a comprehensive response.
        
        Return JSON:
        {{
            "facts": ["fact1", "fact2", "fact3"],
            "entities": ["entity1", "entity2"],
            "summary": "summary of all findings",
            "sources_used": ["source1", "source2"],
            "confidence": 0.0-1.0
        }}
        """
        
        response = self.process(synthesis_prompt)
        synthesis = extract_and_parse_json(response)
        
        # Return complete result
        return {
            "agent": self.name,
            "original_query": original_query,
            "tool_decision": tool_decision,
            "tool_results": tool_results,
            "final_synthesis": synthesis
        }

### Tool-Enabled Sentiment Analyzer

Here's our sentiment analyzer with tool capabilities. It can search for additional context about topics to provide more informed sentiment analysis.

**Use Cases:**
- Analyzing sentiment about products/services by looking up reviews
- Understanding context around events or topics
- Providing more nuanced sentiment analysis with background information

In [108]:
class SentimentAnalyzerAgentToolsEnabled(BaseAgent):
    """Sentiment agent that can decide to use tools"""
    
    def __init__(self):
        super().__init__(
            "SentimentAgent",
            "You are a sentiment analysis agent. You can use web search and Wikipedia tools for context."
        )
        self.tools = ToolRegistry()
    
    def handle_query(self, query: str) -> Dict:
        """Main method: decide tools → execute tools → synthesize"""
        print(f"\n😊 [{self.name}] Received query: {query}")
        
        # Step 1: Decide which tools to use and generate params
        tool_decision = self._decide_tools(query)
        print(f"🔧 [{self.name}] Tool decision: {tool_decision}")
        
        # Step 2: Execute the tools
        tool_results = self._execute_tools(tool_decision)
        print(f"📊 [{self.name}] Tool execution complete")
        
        # Step 3: Synthesize final result
        final_result = self._synthesize_result(query, tool_decision, tool_results)
        print(f"✅ [{self.name}] Result synthesized")
        
        return final_result
    
    def _decide_tools(self, query: str) -> Dict:
        """Step 1: Decide which tools to use and generate parameters"""
        print(f"🧠 [{self.name}] Deciding which tools to use...")
        
        # Get available tools from ToolRegistry
        available_tools = ToolRegistry.get_available_tools()
        
        decision_prompt = f"""
        Query: {query}
        
        Available tools:
        {available_tools}
        
        Decide which tools to use for better sentiment analysis and generate the parameters.
        
        Respond with JSON:
        {{
            "tools_to_use": [
                {{
                    "tool": "tool_name",
                    "params": {{"param1": "value1", "param2": "value2"}}
                }}
            ],
            "reasoning": "why these tools help with sentiment analysis"
        }}
        
        Use empty list [] if no tools needed.
        """
        
        response = self.process(decision_prompt)
        return extract_and_parse_json(response)
    
    def _execute_tools(self, tool_decision: Dict) -> Dict:
        """Step 2: Execute the decided tools with their parameters"""
        print(f"⚡ [{self.name}] Executing tools...")
        
        tool_results = {}
        tools_to_use = tool_decision.get("tools_to_use", [])
        
        for i, tool_call in enumerate(tools_to_use):
            tool_name = tool_call.get("tool")
            params = tool_call.get("params", {})
            
            print(f"🔍 [{self.name}] Executing {tool_name} with params: {params}")
            
            # Use getattr to call tools dynamically
            if hasattr(self.tools, tool_name):
                tool_method = getattr(self.tools, tool_name)
                try:
                    result = tool_method(**params)
                except TypeError as e:
                    result = {"error": f"Invalid parameters for {tool_name}: {e}"}
            else:
                result = {"error": f"Tool {tool_name} not found in ToolRegistry"}
            
            tool_results[f"{tool_name}_{i}"] = result
            print(f"📋 [{self.name}] {tool_name} completed")
        
        return tool_results
    
    def _synthesize_result(self, original_query: str, tool_decision: Dict, tool_results: Dict) -> Dict:
        """Step 3: Synthesize final result from tools and original query"""
        print(f"🧬 [{self.name}] Synthesizing final result...")
        
        synthesis_prompt = f"""
        Original query: {original_query}
        
        Tool decision made: {json.dumps(tool_decision, indent=2)}
        
        Tool results: {json.dumps(tool_results, indent=2)}
        
        Now analyze sentiment using the original query and any context from tools.
        
        Return JSON:
        {{
            "sentiment": "positive/negative/neutral",
            "confidence": 0.0-1.0,
            "tone": ["emotion1", "emotion2"],
            "justification": "why this sentiment",
            "context_influence": "how tool results influenced analysis"
        }}
        """
        
        response = self.process(synthesis_prompt)
        synthesis = extract_and_parse_json(response)
        
        # Return complete result
        return {
            "agent": self.name,
            "original_query": original_query,
            "tool_decision": tool_decision,
            "tool_results": tool_results,
            "final_synthesis": synthesis
        }

## 13. Multi-Agent Orchestration: The Orchestrator

Finally, let's build an **Orchestrator** - a meta-agent that coordinates other agents! This demonstrates the pinnacle of agent architecture.

### The Orchestrator's Role:

1. **Request Analysis**: Understands what the user is asking for
2. **Agent Selection**: Decides which specialized agents to use
3. **Task Delegation**: Assigns work to the appropriate agents
4. **Result Integration**: Combines outputs from multiple agents

### Why This Matters:

- **Scalability**: Easy to add new specialized agents
- **Flexibility**: Can handle complex requests requiring multiple capabilities
- **Maintainability**: Each agent focuses on what it does best
- **User Experience**: Single interface that handles complex multi-step tasks

This pattern is used in many real-world AI systems!

### 🎭 Multi-Agent Orchestration Architecture

```
                    ┌─────────────────────────────────┐
                    │          USER REQUEST           │
                    │   "Facts about iPhone 15 and   │
                    │    how people feel about it"    │
                    └─────────────────┬───────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────┐
│                      ORCHESTRATOR                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Step 1: REQUEST ANALYSIS                                       │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │  • Parse user intent                                    │    │
│  │  • Identify required capabilities                       │    │
│  │  • Determine agent delegation strategy                  │    │
│  └─────────────────────────────────────────────────────────┘    │
│                              │                                  │
│                              ▼                                  │
│  Step 2: AGENT DELEGATION                                       │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │         Decision: Use both agents                       │    │
│  │  ┌─────────────────┐    ┌─────────────────┐             │    │
│  │  │ use_fact_agent  │    │use_sentiment_   │             │    │
│  │  │    = true       │    │   agent = true  │             │    │
│  │  └─────────────────┘    └─────────────────┘             │    │
│  └─────────────────────────────────────────────────────────┘    │
│                              │                                  │
│                              ▼                                  │
│  Step 3: PARALLEL EXECUTION                                     │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │                                                         │    │
│  │  ┌─────────────────────┐  ┌──────────────────────┐      │    │
│  │  │ FactExtractorAgent  │  │SentimentAnalyzerAgent│      │    │
│  │  │   (Tools Enabled)   │  │   (Tools Enabled)    │      │    │
│  │  └─────────────────────┘  └──────────────────────┘      │    │
│  │            │                          │                 │    │
│  │            ▼                          ▼                 │    │
│  │  ┌─────────────────────┐  ┌─────────────────────┐       │    │
│  │  │   Tool Decision     │  │   Tool Decision     │       │    │
│  │  │   Tool Execution    │  │   Tool Execution    │       │    │
│  │  │   Result Synthesis  │  │   Result Synthesis  │       │    │
│  │  └─────────────────────┘  └─────────────────────┘       │    │
│  │            │                          │                 │    │
│  │            ▼                          ▼                 │    │
│  │  ┌─────────────────────┐  ┌─────────────────────┐       │    │
│  │  │   Facts + Sources   │  │ Sentiment + Context │       │    │
│  │  └─────────────────────┘  └─────────────────────┘       │    │
│  └─────────────────────────────────────────────────────────┘    │
│                              │                                  │
│                              ▼                                  │
│  Step 4: RESULT INTEGRATION                                     │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │           Combined Multi-Agent Response                 │    │
│  │  {                                                      │    │
│  │    "request": "original query",                         │    │
│  │    "delegation_decision": {...},                        │    │
│  │    "agent_results": {                                   │    │
│  │      "facts": {agent output with tools},                │    │
│  │      "sentiment": {agent output with tools}             │    │
│  │    }                                                    │    │
│  │  }                                                      │    │
│  └─────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
                    ┌─────────────────────────────────┐
                    │      COMPREHENSIVE RESPONSE     │
                    │   Facts + Sentiment + Sources   │
                    │     from Multiple Agents        │
                    └─────────────────────────────────┘

ONE REQUEST → MULTIPLE SPECIALIZED AGENTS → INTEGRATED RESPONSE
```

In [109]:
class Orchestrator(BaseAgent):
    """Simple orchestrator that delegates to agents"""
    
    def __init__(self):
        super().__init__(
            "Orchestrator",
            "You decide which agents to use for different queries."
        )
        
        self.fact_agent = FactExtractorAgentToolsEnabled()
        self.sentiment_agent = SentimentAnalyzerAgentToolsEnabled()
        
        print(f"🎯 [{self.name}] Ready with fact and sentiment agents")
    
    def handle_request(self, user_request: str) -> Dict:
        """Delegate to appropriate agents"""
        print(f"\n🎬 [{self.name}] Processing: {user_request}")
        
        # Decide which agents to use
        delegation = self._decide_delegation(user_request)
        print(f"📋 [{self.name}] Delegation: {delegation}")
        
        # Execute delegation
        results = {}
        
        if delegation.get("use_fact_agent"):
            print(f"\n🎯 [{self.name}] Delegating to fact agent...")
            results["facts"] = self.fact_agent.handle_query(user_request)
        
        if delegation.get("use_sentiment_agent"):
            print(f"\n🎯 [{self.name}] Delegating to sentiment agent...")
            results["sentiment"] = self.sentiment_agent.handle_query(user_request)
        
        return {
            "request": user_request,
            "delegation_decision": delegation,
            "agent_results": results
        }
    
    def _decide_delegation(self, request: str) -> Dict:
        """Decide which agents to use"""
        delegation_prompt = f"""
        Request: {request}
        
        Available agents:
        - fact_agent: Extracts facts, entities, statistics
        - sentiment_agent: Analyzes sentiment, emotions, tone
        
        Which agents should handle this request?
        
        JSON response:
        {{
            "use_fact_agent": true/false,
            "use_sentiment_agent": true/false,
            "reasoning": "why this delegation"
        }}
        """
        
        response = self.process(delegation_prompt)
        return extract_and_parse_json(response)

## Testing Our Complete System

Let's test our complete multi-agent system! First, let's see what tools are available to our agents:

In [110]:
print(ToolRegistry.get_available_tools())

- web_search(query: str, max_results: int = 5) - Performs a web search using Brave Search API and returns a summary of the top results.
- wikipedia_search(query: str, sentences: int = 3) - Searches Wikipedia for a query and returns a summary and URL of the page.


### Initialize the Orchestrator

Now let's create our orchestrator that will coordinate all our specialized agents:

In [111]:
orchestrator = Orchestrator()

🎯 [Orchestrator] Ready with fact and sentiment agents


### Complex Test Queries

Let's test our system with complex queries that require multiple types of analysis. These queries will demonstrate how the orchestrator intelligently delegates work to different agents:

In [112]:
test_queries = [
    "What are the facts about the iPhone 15 and how do people feel about it?",
    "Analyze this review: 'Tesla Model Y is amazing! Great range and autopilot features. Worth every penny!'",
    "Tell me about recent developments in AI safety",
]

### Running the Complete System Test

Now let's run our comprehensive test! Watch how the orchestrator:

1. **Analyzes each query** to understand what's needed
2. **Delegates to appropriate agents** (fact extraction, sentiment analysis, or both)
3. **Coordinates tool usage** when agents need external information
4. **Integrates results** into a cohesive response

This demonstrates a fully functional multi-agent system in action!

In [113]:
for i, query in enumerate(test_queries, 1):
        print(f"\n🧪 TEST CASE {i}")
        print("=" * 50)
        
        result = orchestrator.handle_request(query)
        
        print(f"\n📊 FINAL RESULTS:")
        for agent_type, agent_result in result["agent_results"].items():
            print(f"\n{agent_type.upper()}:")
            if "final_synthesis" in agent_result:
                synthesis = agent_result["final_synthesis"]
                print(f"  Summary: {synthesis.get('summary', 'N/A')}")
                print(f"  Confidence: {synthesis.get('confidence', 'N/A')}")
        
        print("\n" + "=" * 50)


🧪 TEST CASE 1

🎬 [Orchestrator] Processing: What are the facts about the iPhone 15 and how do people feel about it?
📋 [Orchestrator] Delegation: {'use_fact_agent': True, 'use_sentiment_agent': True, 'reasoning': "The query explicitly asks for facts about the iPhone 15, which falls under the fact_agent's purview. It also asks how people feel about the iPhone 15, which is a sentiment analysis task for the sentiment_agent."}

🎯 [Orchestrator] Delegating to fact agent...

🤖 [FactAgent] Received query: What are the facts about the iPhone 15 and how do people feel about it?
🧠 [FactAgent] Deciding which tools to use...
📋 [Orchestrator] Delegation: {'use_fact_agent': True, 'use_sentiment_agent': True, 'reasoning': "The query explicitly asks for facts about the iPhone 15, which falls under the fact_agent's purview. It also asks how people feel about the iPhone 15, which is a sentiment analysis task for the sentiment_agent."}

🎯 [Orchestrator] Delegating to fact agent...

🤖 [FactAgent] Received

## Conclusion: Your Journey into LLM Agents

Congratulations! You've built a complete multi-agent system from scratch. Let's recap what we've learned:

### Key Concepts Covered:

1. **Basic Agent Architecture**: LLM + Instructions = Agent
2. **Specialized Agents**: Different prompts create different capabilities
3. **Structured Output**: JSON responses for programmatic processing
4. **Agent Coordination**: Independent vs. Sequential processing patterns
5. **Tool Integration**: Extending agents with external capabilities
6. **Multi-Agent Orchestration**: Coordinating multiple specialized agents

### 🏗️ Complete System Architecture Evolution

```
LEVEL 1: Basic Agent
┌─────────────────┐
│   BaseAgent     │
│ LLM+Instructions│
└─────────────────┘
         ↓

LEVEL 2: Specialized Agents
┌─────────────────┐ ┌─────────────────┐ ┌─────────────────┐
│ FactExtractor   │ │SentimentAnalyzer│ │   Summarizer    │
│                 │ │                 │ │                 │
└─────────────────┘ └─────────────────┘ └─────────────────┘
         ↓

LEVEL 3: Structured Output Agents
┌─────────────────────────────────────────────────────────────┐
│  Advanced Agents with JSON Output + Error Handling          │
│  ┌──────────────────┐ ┌──────────────────────┐              │
│  │FactExtractorAgent│ │SentimentAnalyzerAgent│              │
│  │  → JSON Output   │ │  → JSON Output       │              │
│  └──────────────────┘ └──────────────────────┘              │
└─────────────────────────────────────────────────────────────┘
         ↓

LEVEL 4: Tool-Enabled Agents
┌─────────────────────────────────────────────────────────────┐
│              Agents + External Tools                        │
│  ┌─────────────────────────────────────────────────────┐    │
│  │               ToolRegistry                          │    │
│  │  ┌─────────────────┐ ┌──────────────────┐           │    │
│  │  │  web_search()   │ │wikipedia_search()│           │    │
│  │  └─────────────────┘ └──────────────────┘           │    │
│  └─────────────────────────────────────────────────────┘    │
│                              │                              │
│  ┌─────────────────────────────────────────────────────┐    │
│  │  FactExtractorAgentToolsEnabled                     │    │
│  │  SentimentAnalyzerAgentToolsEnabled                 │    │
│  │  → DECIDE → EXECUTE → SYNTHESIZE                    │    │
│  └─────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────┘
         ↓

LEVEL 5: Multi-Agent Orchestration
┌─────────────────────────────────────────────────────────────┐
│                    ORCHESTRATOR                             │
│  ┌─────────────────────────────────────────────────────┐    │
│  │  User Request Analysis & Agent Delegation           │    │
│  └─────────────────────────────────────────────────────┘    │
│                              │                              │
│  ┌─────────────────────┐    ┌──────────────────────┐        │
│  │ FactExtractorAgent  │    │SentimentAnalyzerAgent│        │
│  │   (Tools Enabled)   │    │   (Tools Enabled)    │        │
│  └─────────────────────┘    └──────────────────────┘        │
│                              │                              │
│  ┌─────────────────────────────────────────────────────┐    │
│  │           Integrated Response                       │    │
│  └─────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────┘

FROM SIMPLE AGENTS → SOPHISTICATED MULTI-AGENT SYSTEM
```

### Architecture Patterns:

- **BaseAgent**: Foundation for all agents
- **ToolRegistry**: Centralized tool management
- **Three-Step Tool Usage**: Decide → Execute → Synthesize
- **Orchestrator Pattern**: Meta-agent for coordination

### Real-World Applications:

- **Customer Service**: Different agents for different types of inquiries
- **Content Analysis**: Fact-checking, sentiment analysis, summarization
- **Research Assistant**: Information gathering and synthesis
- **Decision Support**: Multi-perspective analysis of complex problems

### Next Steps:

1. **Add More Tools**: Database access, APIs, file processing
2. **Implement Memory**: Let agents remember previous interactions
3. **Add Validation**: Error checking and response quality assurance
4. **Scale Up**: Handle multiple users and concurrent requests
5. **Specialized Domains**: Create agents for specific industries or use cases

### Best Practices Learned:

- **Clear Instructions**: Specific prompts lead to better results
- **Structured Output**: JSON makes agents more useful
- **Error Handling**: Always plan for failures
- **Modular Design**: Keep agents focused on specific tasks
- **Tool Integration**: External capabilities multiply agent power

You now have the foundation to build sophisticated LLM agent systems for any domain!
